# YOLOV8( Using Our First Pretrained Model )
** YOLOv8 is  a advanced CNN- based model that used for object detection and classifications , we will use its classification head that is trained on imagenet with pretrained wieghts , famous for its speed and accuracy in object detection **


* Goal: We’re going to build a production-grade food classification system using the classification head of YOLOv8 (yolov8n-cls.pt). The model will be fine-tuned on 5 selected classes from the Food-101 dataset — pizza, grilled_chicken, sushi, ice_cream, and hamburger. We'll enhance the pretrained model using advanced training techniques to boost its performance.

* Albumentations : Advanced image augementations library that gives complex transformations 

* Cosine Annealing LR : Learning rate scheduler that just LR smoothly 

* Model Ensembling: Combines multiple mdels predictons for higher accuracy

* we will take 5 classes ( pizza, grilled_chicken , sushi , ice_cream , hamburger)

# 
├── Food_Classification_YOLOv8_Module12.ipynb
├── generate_yaml.py
├── food-101/
│   ├── images/
│   │   ├── pizza/
│   │   ├── grilled_chicken/
│   │   ├── sushi/
│   │   ├── ice_cream/
│   │   ├── hamburger/
│   ├── meta/
│   ├── food101.yaml



* Make sure to install this before writing the code 

pip install torch==2.4.1 torchvision==0.19.1 opencv-python-headless==4.10.0 ultralytics==8.2.28 albumentations==1.4.8

*  The project uses the Food-101 dataset, a large-scale benchmark dataset for food classification. 

In [ ]:
pip install torch==2.4.1 torchvision==0.19.1 opencv-python-headless==4.10.0 ultralytics==8.2.28 albumentations==1.4.8

# This is the basic setup of any model training 

* I generally use pytorch so I can understand what exactly is happening in the code 

In [ ]:
import torch 
import torch.nn as nn 
import torch.optim as optim 
from torch.optim.lr_scheduler import CosineAnnealingLR
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset, Subset 
import cv2
import numpy as np
import logging 
import os 
import albumentations as A
from typing import List, Optional, Dict, Tuple

# logging setup  to track executions and errors

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s',
                    handlers = [
                        logging.StreamHandler(), #to show logs in console 
                        logging.FileHandler('yolo_training.log') # save logs to a file
                    ] )

logger = logging.getLogger(__name__)

# configurations class for project settings 

class Config:
    def __init__(self):
        self.data_dir = "./foof/food-101"
        self.classes = ['pizza','grilled_chicken','sushi','ice_cream','hamburger']
        self.max_images_per_class = 100
        self.test_images_per_class = 20
        self.epochs = 18
        self.batch_size = 5 # batch size optimized for gpu 
        self.img_size = 224 # image size for yolo v8 
        self.mode_path_nano = "yolov8n_food_classifier.pt" #nano model save path 
        self.model_path_small = "yolov8s_food_classifier.pt" #small model save path 
        self.yaml_path = os.path.join(self.data_dir, 'food101.yaml')
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.lr = 0.001
        logging.info(f"Using device : {self.device}")
       